In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dateutil import parser
from datetime import datetime
import json
from IPython.display import display
import statistics

In [2]:
df1 = pd.read_csv("../dataset/TRAIN_Consumptions.csv")
df1['MEASUREMENT_DATE'] = pd.to_datetime(df1['MEASUREMENT_DATE'])

In [3]:
df2 = pd.read_csv("../dataset/TRAIN_PowerThefts.csv")

In [4]:
df2 = df2.sort_values(by='ACCT_NBR')

In [5]:
thefts_df = df2.copy()
thefts_df['INITIAL_DETECTION_DATE'] = pd.to_datetime(thefts_df['INITIAL_DETECTION_DATE'])

In [6]:
valid_thefts = []

In [7]:
for k, theft in thefts_df.iterrows():
    acct = theft.ACCT_NBR
    succ = theft.SUCCESSOR
    date = theft.INITIAL_DETECTION_DATE
    
    df = df1[df1.ACCT_NBR == acct].set_index('MEASUREMENT_DATE').sort_index()
    
    df = df[df.SUCCESSOR == succ]
    
    if df.empty:
        continue
    
    if pd.to_datetime(df.first_valid_index()) < date:
            valid_thefts.append(theft)

In [8]:
valid_thefts_df = pd.DataFrame(valid_thefts, columns=['ACCT_NBR', 'SUCCESSOR', 'INITIAL_DETECTION_DATE'])

In [9]:
valid_thefts_df.head()

,ACCT_NBR,SUCCESSOR,INITIAL_DETECTION_DATE
1918,348,7,2021-11-18
1920,928,4,2018-04-23
2,957,2,2018-12-18
1923,1402,2,2019-12-27
6,2333,3,2021-05-19


In [25]:
df1.head()

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,329855,1,13,0,56,2019-08-26
1,113886,1,11,1,2236,2022-03-10
2,219271,1,13,2,678,2021-04-20
3,245307,1,13,3,156,2022-04-27
4,250806,1,13,4,502,2022-06-03


In [45]:
merged_df = pd.merge(valid_thefts_df, df1, on=['ACCT_NBR', 'SUCCESSOR'])

In [46]:
new_df = merged_df.groupby(['ACCT_NBR', 'SUCCESSOR']).filter(lambda x: len(x) > 2)

In [47]:
new_df.ACCT_NBR.nunique()

1509

In [39]:
valid_thefts_df.ACCT_NBR.nunique()

1569

In [48]:
new_df = new_df.sort_values(by=['ACCT_NBR', 'SUCCESSOR', 'MEASUREMENT_DATE'])
new_df.head()

,ACCT_NBR,SUCCESSOR,INITIAL_DETECTION_DATE,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
11,348,7,2021-11-18,10,356733,0,2018-01-18
4,348,7,2021-11-18,10,356733,0,2018-05-18
6,348,7,2021-11-18,10,356733,0,2018-09-18
2,348,7,2021-11-18,10,356733,0,2019-01-17
12,348,7,2021-11-18,10,356733,0,2019-05-20


In [61]:
request_df = pd.read_csv("../dataset/Train_Requests.csv")
request_df = request_df.drop(['COMPL_REQUEST_STATUS', 'COMPLETION_DATE'], axis=1)

In [62]:
request_df['REQUEST_TYPE'] = request_df['REQUEST_TYPE'].fillna('unknown')

In [63]:
request_df.head()

,ACCT_NBR,SUCCESSOR,REQUEST_TYPE,REQUEST_DATE
0,493073,7,recon,2019-02-12
1,15120,3,unknown,2015-05-26
2,15120,4,unknown,2017-02-06
3,15120,5,discon,2018-06-28
4,15120,5,reprChange,2018-09-27


In [67]:
new_df['MEASUREMENT_DATE'] = pd.to_datetime(new_df['MEASUREMENT_DATE'])
request_df['REQUEST_DATE'] = pd.to_datetime(request_df['REQUEST_DATE'])

In [64]:
results_df = pd.DataFrame(columns=['ACCT_NBR', 'SUCCESSOR', 'REQUEST_TYPE', 'REQUEST_COUNT', 'DAYS_SINCE_LAST_MEASUREMENT'])


In [65]:
grouped_requests.head()

,ACCT_NBR,SUCCESSOR,REQUEST_TYPE,REQUEST_COUNT
0,1,4,discon,2
1,1,4,recon,1
2,1,4,reprPause,1
3,1,5,discon,2
4,1,5,recon,2


In [13]:
merged = pd.merge(valid_thefts_df, df1, on='ACCT_NBR', how='left', suffixes=('_df1', '_df2'))

In [14]:
result = merged[~((merged['SUCCESSOR_df1'] == merged['SUCCESSOR_df2']) & merged['SUCCESSOR_df2'].notna())]

In [15]:
result.head()

,ACCT_NBR,SUCCESSOR_df1,INITIAL_DETECTION_DATE,SUCCESSOR_df2,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,348,7,2021-11-18,8,10,356733,0,2021-10-01
5,348,7,2021-11-18,9,10,356733,781,2022-05-19
6,348,7,2021-11-18,8,10,356733,944,2022-03-08
8,348,7,2021-11-18,9,10,356733,2172,2022-09-20
11,348,7,2021-11-18,9,10,356733,0,2022-03-09


In [16]:
result = result.sort_values(['ACCT_NBR', 'SUCCESSOR_df2'])

In [21]:
result.loc[result.ACCT_NBR == 487974]

,ACCT_NBR,SUCCESSOR_df1,INITIAL_DETECTION_DATE,SUCCESSOR_df2,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
34596,487974,2,2019-08-11,3,10,455426,0,2020-01-31


In [23]:
df1.loc[df1.ACCT_NBR == 487974]

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
2098797,487974,2,10,455426,0,2019-06-06
2945261,487974,3,10,455426,0,2020-01-31
4982531,487974,2,10,455426,0,2019-09-28


In [24]:
valid_thefts_df.loc[valid_thefts_df.ACCT_NBR == 487974]

,ACCT_NBR,SUCCESSOR,INITIAL_DETECTION_DATE
1864,487974,2,2019-08-11
